In [1]:
import pandas as pd
import uuid
from datetime import datetime
import numpy as np
from grakn.client import GraknClient
from tqdm import tqdm
tqdm.pandas()

In [2]:
df=pd.read_excel(r"C:\Users\SRIKANTH\Desktop\OpenCTI_Data.xlsx",sheet_name='intrusion-set')

In [3]:
df.keys()

Index(['internal_id_key', 'stix_id_key', 'stix_label', 'entity_type',
       'created', 'modified', 'revoked', 'object_status', 'created_at',
       'created_at_day', 'created_at_month', 'created_at_year', 'updated_at',
       'name', 'description', 'alias', 'graph_data', 'first_seen',
       'first_seen_day', 'first_seen_month', 'first_seen_year', 'last_seen',
       'last_seen_day', 'last_seen_month', 'last_seen_year', 'goal',
       'sophistication', 'resource_level', 'primary_motivation',
       'secondary_motivation', 'type', 'id', 'labels', 'external_references',
       'object_marking_refs', 'granular_markings'],
      dtype='object')

In [4]:
def  insert_general_info(df,ent_type,create):
    if create:
        for index,i in df.iterrows():
            df.internal_id_key[index]=str(uuid.uuid4())
            df.stix_id_key[index]=ent_type+'--'+str(uuid.uuid4())
            df.stix_label[index]='NA'
            df.entity_type[index]=ent_type
            df.created=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at_day=(datetime.now()).strftime("%Y-%m-%d")
            df.created_at_month=(datetime.now()).strftime("%Y-%m")
            df.created_at_year=(datetime.now()).strftime("%Y")
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.revoked=False
            df.object_status='NA'
    else:
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
    return df
    

In [5]:
result=insert_general_info(df=df,ent_type='Intrusion-set',create=True)
result.fillna('NA',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\SRIKANTH\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithC

In [12]:
def insert_one_intrusion_set(df,session):
    if df.first_seen != 'NA':
        f=df.first_seen.strftime("%Y-%m-%dT%H:%M:%S")
        f_m=df.first_seen.strftime("%Y-%m")
        f_d=df.first_seen.strftime("%Y-%m-%d")
    elif df.first_seen_month != 'NA':
        f_m=df.first_seen_month
    elif df.first_seen_day != 'NA':
        f_d=df.first_seen_day
    else:
        f_d=None
        f_m=None
        f=None
               
    aliases=df.alias.split('\n')

    graql_match_query = f"match $i isa Intrusion-Set, " \
                        f"has name \"{df['name']}_\"; " \
                        f"get;"
    # print(graql_match_query)
    with session.transaction().read() as read_transaction:
        # read_transaction.query(graql_match_query)
        answer_iterator = None
        answer_iterator = read_transaction.query(graql_match_query, infer=True)
        data = answer_iterator.collect_concepts()
                
        if len(data) > 0:
            # Update data.
            # Columns to be updated: object_status, revoked, updated_at, modified, description, alias
            # print("Update intrusion-set...")
            print("Update: " + df['name'])
            graql_attribute_delete_query = f"match $i isa Intrusion-Set, " \
                                           f"has name \"{df['name']}_\", "\
                                           f"has alias $a via $i1, "\
                                           f"has description $a via $i2; "\
                                           f"delete $i1, $i2;"
            update_flg = False
            first_attr_flg = True
            graql_attribute_update_query = f"match $i isa Intrusion-Set, " \
                                           f"has name \"{df['name']}_\"; insert $i"
            if df['description'] != 'NA':
                # graql_attribute_delete_query += f" has description $a via $i2; "\
                graql_attribute_update_query += f" has description \"{df['description']}\" "
                if first_attr_flg:
                    graql_attribute_update_query += ","
                update_flg = True
                first_attr_flg = False
            if len(aliases) > 0:
                s = "|"
                graql_attribute_update_query += f" has alias \"{s.join(aliases)}\" " 
                if first_attr_flg:
                    graql_attribute_update_query += ","
                update_flg = True
                first_attr_flg = False
            graql_attribute_update_query += ";"
            # print(graql_attribute_update_query)
            if update_flg:
                with session.transaction().write() as write_transaction:
                    # make a write transection with the query
                    write_transaction.query(graql_attribute_delete_query)
                    write_transaction.query(graql_attribute_update_query)
                    # remember to commit at the end
                    write_transaction.commit()           
        else:
            # print("Insert intrusion-set...")
            print("Insert: " + df['name'])
            graql_insert_query = f"insert $x isa Intrusion-Set, " \
                                 f"has internal_id_key \"{df['internal_id_key']}\", " \
                                 f"has stix_id_key \"{df['stix_id_key']}\", " \
                                 f"has stix_label \"{df['stix_label']}\", " \
                                 f"has entity_type \"{df['entity_type']}\", " \
                                 f"has created {df['created']}, " \
                                 f"has revoked \"{df['revoked']}\", "\
                                 f"has created_at {df['created_at']}, " \
                                 f"has created_at_day \"{df['created_at_day']}\", " \
                                 f"has created_at_month \"{df['created_at_month']}\", " \
                                 f"has created_at_year \"{df['created_at_year']}\", "\
                                 f"has updated_at {df['updated_at']}, "\
                                 f"has name \"{df['name']}_\" "
                                 
            if df['description'] != 'NA':
                graql_insert_query += f", has description \"{df['description']}\" "
            if len(aliases) > 0:
                s = "|"
                graql_insert_query += f", has alias \"{s.join(aliases)}\" "
            if f is not None:
                graql_insert_query += f", has first_seen {f} "
            if f_d is not None:
                graql_insert_query += f", has first_seen_day \"{f_d}\" "
            if f_m is not None:
                graql_insert_query += f", has first_seen_month \"{f_m}\" "
            if f_m is not None:
                graql_insert_query += f", has first_seen_year \"{df['first_seen_year']}\""
            graql_insert_query += ";"
            # print(graql_insert_query)
            with session.transaction().write() as write_transaction:
                # make a write transection with the query
                write_transaction.query(graql_insert_query)
                # remember to commit at the end
                write_transaction.commit()

In [7]:
def load_data_into_grakn(session,input_df):
    print("Inserting intrusion-set...")
    # using progress_apply instead of apply so we have a progress bar form tqdm
    # input_df.progress_apply(insert_one_intrusion_set, axis=1, session=session)
    print("Loading intrusion-set...")
    for index,row in input_df.iterrows():
        insert_one_intrusion_set(df=row,session=session)

In [8]:
def build_grakn_graph(input_df, keyspace_name):
    with GraknClient(uri="192.168.83.128:48555") as client:
        with client.session(keyspace = keyspace_name) as session:
            load_data_into_grakn(session,input_df)

In [13]:
build_grakn_graph(input_df=result,keyspace_name='grakn')

Inserting intrusion-set...
Loading intrusion-set...
Update: APT12
Update: APT28
Update: BlackOasis
Update: Cobalt
Update: APT30
Update: APT12
